In [19]:
import PIL.ImageOps
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image


def grand_sum(image: Image):
    return np.sum(np.array(image) >= 200)


df = pd.read_csv("datasets/query_list.csv")
dataset_path = "datasets/deep-fashion/In-shop Clothes Retrieval Benchmark/Img/"


def calculate_difference(path: str):
    gt_mask_path = (".".join(path.split(".")[:-1])) + "_mask.png"
    mask_path = (".".join(path.split(".")[:-1])) + "_mask_sam.png"

    M_groundtruth_path = dataset_path + gt_mask_path
    M_original_path = dataset_path + mask_path

    M_groundtruth = Image.open(M_groundtruth_path).convert("RGB")
    M_original = PIL.ImageOps.invert(Image.open(M_original_path).convert("RGB"))
    difference = np.array(M_original) - np.array(M_groundtruth)
    return grand_sum(difference) / grand_sum(M_groundtruth)


df["difference"] = df["image_name"].apply(calculate_difference)
df

,Unnamed: 0,image_name,difference
0,0,img_highres/MEN/Jackets_Vests/id_00000084/08_1...,0.000650
1,1,img_highres/MEN/Jackets_Vests/id_00000094/01_1...,0.000379
2,2,img_highres/MEN/Jackets_Vests/id_00000094/04_1...,0.001158
3,3,img_highres/MEN/Sweaters/id_00000145/01_1_fron...,0.001360
4,4,img_highres/MEN/Shirts_Polos/id_00000193/04_1_...,0.000208
...,...,...,...
521,521,img_highres/WOMEN/Tees_Tanks/id_00007962/07_1_...,0.917994
522,522,img_highres/WOMEN/Tees_Tanks/id_00007962/07_2_...,1.566279
523,523,img_highres/MEN/Tees_Tanks/id_00007967/02_2_si...,0.504207
524,524,img_highres/WOMEN/Tees_Tanks/id_00007969/03_1_...,0.458658


In [24]:
df_filtered = df[df['difference'] < 0.05]

In [36]:
df_filtered["is_man"] = False
for index, row in tqdm(df_filtered.iterrows()):
    is_woman = "WOMEN" in row["image_name"]
    df_filtered["is_man"][index] = not is_woman

/tmp/ipykernel_14149/3734725119.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["is_man"] = False
0it [00:00, ?it/s]/tmp/ipykernel_14149/3734725119.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["is_man"][index] = not is_woman
/tmp/ipykernel_14149/3734725119.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["is_man"][index] = not is_

In [48]:
len_MEN = len(df_filtered[df_filtered["is_man"] == True])
len_WOMEN = len(df_filtered[df_filtered["is_man"] == False])
print("MEN", len_MEN)
print("WOMEN", len_WOMEN)
more_men = len_MEN > len_WOMEN
amount_to_ignore = len_MEN - len_WOMEN if more_men else len_WOMEN - len_MEN
for index, row in tqdm(df_filtered.iterrows()):
    if amount_to_ignore > 0 and (row["is_man"] if more_men else not row["is_man"]):
        df_filtered = df_filtered.drop(index)
        amount_to_ignore -= 1
len_MEN = len(df_filtered[df_filtered["is_man"] == True])
len_WOMEN = len(df_filtered[df_filtered["is_man"] == False])
print("MEN", len_MEN)
print("WOMEN", len_WOMEN)
test_dataset = pd.DataFrame()
test_dataset["image_name"] = df_filtered["image_name"]
test_dataset = test_dataset.reset_index(drop=True)
test_dataset

MEN 208
WOMEN 208


416it [00:00, 48114.67it/s]

MEN 208
WOMEN 208


,image_name
0,img_highres/MEN/Tees_Tanks/id_00000822/01_1_fr...
1,img_highres/MEN/Sweatshirts_Hoodies/id_0000091...
2,img_highres/MEN/Jackets_Vests/id_00000946/04_1...
3,img_highres/MEN/Sweatshirts_Hoodies/id_0000095...
4,img_highres/MEN/Sweaters/id_00001008/04_1_fron...
...,...
411,img_highres/WOMEN/Tees_Tanks/id_00007923/04_1_...
412,img_highres/WOMEN/Blouses_Shirts/id_00007936/0...
413,img_highres/WOMEN/Tees_Tanks/id_00007958/04_2_...
414,img_highres/WOMEN/Tees_Tanks/id_00007958/05_1_...


In [49]:
test_dataset.to_csv("datasets/query_list.csv")
test_dataset

,image_name
0,img_highres/MEN/Tees_Tanks/id_00000822/01_1_fr...
1,img_highres/MEN/Sweatshirts_Hoodies/id_0000091...
2,img_highres/MEN/Jackets_Vests/id_00000946/04_1...
3,img_highres/MEN/Sweatshirts_Hoodies/id_0000095...
4,img_highres/MEN/Sweaters/id_00001008/04_1_fron...
...,...
411,img_highres/WOMEN/Tees_Tanks/id_00007923/04_1_...
412,img_highres/WOMEN/Blouses_Shirts/id_00007936/0...
413,img_highres/WOMEN/Tees_Tanks/id_00007958/04_2_...
414,img_highres/WOMEN/Tees_Tanks/id_00007958/05_1_...
